### Import needed libraries

In [1]:
import numpy as np
import pandas as pd
import requests
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize

print('Libraries imported.')

Libraries imported.


### Import file from previous task

In [2]:
toronto_geo_data = pd.read_csv('Toronto_geo_data_Task_2.csv', index_col=0)

toronto_geo_data.head()

Latitude  Longitude        Borough  \
Postal Code                                        
M1B          43.806686 -79.194353  Scarborough\n   
M1C          43.784535 -79.160497  Scarborough\n   
M1E          43.763573 -79.188711  Scarborough\n   
M1G          43.770992 -79.216917  Scarborough\n   
M1H          43.773136 -79.239476  Scarborough\n   

                                       Neighborhood  
Postal Code                                          
M1B                                  Malvern, Rouge  
M1C          Rouge Hill, Port Union, Highland Creek  
M1E               Guildwood, Morningside, West Hill  
M1G                                          Woburn  
M1H                                       Cedarbrae

In [3]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_geo_data['Borough'].unique()),
        toronto_geo_data.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [4]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="can_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [5]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_geo_data['Latitude'], toronto_geo_data['Longitude'], toronto_geo_data['Borough'], toronto_geo_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [6]:
CLIENT_ID = '50JY4UUV0HNGDBG1L5N2VE3AWFAVAXG10J2FB4AUCWHM3Q1K' 
CLIENT_SECRET = '4RN3DUDQMHRWBINPNSVO3QQ5T2Z4GGMXEVHCUSJPZHVCU3H5'
VERSION = '20191101'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 50JY4UUV0HNGDBG1L5N2VE3AWFAVAXG10J2FB4AUCWHM3Q1K
CLIENT_SECRET:4RN3DUDQMHRWBINPNSVO3QQ5T2Z4GGMXEVHCUSJPZHVCU3H5


### Creating a function to repeat the same process to all the neighborhoods in Toronto

In [7]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [8]:
toronto_venues = getNearbyVenues(names=toronto_geo_data['Neighborhood'],
                                   latitudes=toronto_geo_data['Latitude'],
                                   longitudes=toronto_geo_data['Longitude'],
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

In [9]:
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                             
Agincourt                                                            5   
Alderwood, Long Branch                                               7   
Bathurst Manor, Wilson Heights, Downsview North                     21   
Bayview Village                                                      4   
Bedford Park, Lawrence Manor East                                   22   
...                                                                ...   
Willowdale, Willowdale West                                          5   
Woburn                                                               4   
Woodbine Heights                                                     8   
York Mills West                                                      2   
York Mills, Silver Hills                                             1   

                                                 Neighborhood Longitude  \
Neighborhood                                                              
Agincourt                                                             5   
Alderwood, Long Branch                                                7   
Bathurst Manor, Wilson Heights, Downsview North                      21   
Bayview Village                                                       4   
Bedford Park, Lawrence Manor East                                    22   
...                                                                 ...   
Willowdale, Willowdale West                                           5   
Woburn                                                                4   
Woodbine Heights                                                      8   
York Mills West                                                       2   
York Mills, Silver Hills                                              1   

                                                 Venue  Venue Latitude  \
Neighborhood                                                             
Agincourt                                            5               5   
Alderwood, Long Branch                               7               7   
Bathurst Manor, Wilson Heights, Downsview North     21              21   
Bayview Village                                      4               4   
Bedford Park, Lawrence Manor East                   22              22   
...                                                ...             ...   
Willowdale, Willowdale West                          5               5   
Woburn                                               4               4   
Woodbine Heights                                     8               8   
York Mills West                                      2               2   
York Mills, Silver Hills                             1               1   

                                                 Venue Longitude  \
Neighborhood                                                       
Agincourt                                                      5   
Alderwood, Long Branch                                         7   
Bathurst Manor, Wilson Heights, Downsview North               21   
Bayview Village                                                4   
Bedford Park, Lawrence Manor East                             22   
...                                                          ...   
Willowdale, Willowdale West                                    5   
Woburn                                                         4   
Woodbine Heights                                               8   
York Mills West                                                2   
York Mills, Silver Hills                                       1   

                                                 Venue Category  
Neighborhood                                                     
Agincourt                                                     5  
Alderwood, Long Branch                                        7 

In [10]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 273 uniques categories.


### Analyse Neighborhoods

In [11]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

Yoga Studio  Accessories Store  Afghan Restaurant  Airport  \
0            0                  0                  0        0   
1            0                  0                  0        0   
2            0                  0                  0        0   
3            0                  0                  0        0   
4            0                  0                  0        0   

   Airport Food Court  Airport Gate  Airport Lounge  Airport Service  \
0                   0             0               0                0   
1                   0             0               0                0   
2                   0             0               0                0   
3                   0             0               0                0   
4                   0             0               0                0   

   Airport Terminal  American Restaurant  ...  Trail  Train Station  \
0                 0                    0  ...      0              0   
1                 0                    0  ...      0              0   
2                 0                    0  ...      0              0   
3                 0                    0  ...      0              0   
4                 0                    0  ...      0              0   

   Turkish Restaurant  Vegetarian / Vegan Restaurant  Video Game Store  \
0                   0                              0                 0   
1                   0                              0                 0   
2                   0                              0                 0   
3                   0                              0                 0   
4                   0                              0                 0   

   Vietnamese Restaurant  Warehouse Store  Wine Bar  Wings Joint  \
0                      0                0         0            0   
1                      0                0         0            0   
2                      0                0         0            0   
3                      0                0         0            0   
4                      0                0         0            0   

   Women's Store  
0              0  
1              0  
2              0  
3              0  
4              0  

[5 rows x 273 columns]

In [12]:
toronto_onehot.shape

(2139, 273)

In [13]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

Neighborhood  Yoga Studio  \
0                                         Agincourt          0.0   
1                            Alderwood, Long Branch          0.0   
2   Bathurst Manor, Wilson Heights, Downsview North          0.0   
3                                   Bayview Village          0.0   
4                 Bedford Park, Lawrence Manor East          0.0   
..                                              ...          ...   
91                      Willowdale, Willowdale West          0.0   
92                                           Woburn          0.0   
93                                 Woodbine Heights          0.0   
94                                  York Mills West          0.0   
95                         York Mills, Silver Hills          0.0   

    Accessories Store  Afghan Restaurant  Airport  Airport Food Court  \
0                 0.0                0.0      0.0                 0.0   
1                 0.0                0.0      0.0                 0.0   
2                 0.0                0.0      0.0                 0.0   
3                 0.0                0.0      0.0                 0.0   
4                 0.0                0.0      0.0                 0.0   
..                ...                ...      ...                 ...   
91                0.0                0.0      0.0                 0.0   
92                0.0                0.0      0.0                 0.0   
93                0.0                0.0      0.0                 0.0   
94                0.0                0.0      0.0                 0.0   
95                0.0                0.0      0.0                 0.0   

    Airport Gate  Airport Lounge  Airport Service  Airport Terminal  ...  \
0            0.0             0.0              0.0               0.0  ...   
1            0.0             0.0              0.0               0.0  ...   
2            0.0             0.0              0.0               0.0  ...   
3            0.0             0.0              0.0               0.0  ...   
4            0.0             0.0              0.0               0.0  ...   
..           ...             ...              ...               ...  ...   
91           0.0             0.0              0.0               0.0  ...   
92           0.0             0.0              0.0               0.0  ...   
93           0.0             0.0              0.0               0.0  ...   
94           0.0             0.0              0.0               0.0  ...   
95           0.0             0.0              0.0               0.0  ...   

    Trail  Train Station  Turkish Restaurant  Vegetarian / Vegan Restaurant  \
0     0.0            0.0                 0.0                            0.0   
1     0.0            0.0                 0.0                            0.0   
2     0.0            0.0                 0.0                            0.0   
3     0.0            0.0                 0.0                            0.0   
4     0.0            0.0                 0.0                            0.0   
..    ...            ...                 ...                            ...   
91    0.0            0.0                 0.0                            0.0   
92    0.0            0.0                 0.0                            0.0   
93    0.0            0.0                 0.0                            0.0   
94    0.0            0.0                 0.0                            0.0   
95    0.0            0.0                 0.0                            0.0   

    Video Game Store  Vietnamese Restaurant  Warehouse Store  Wine Bar  \
0                0.0                    0.0              0.0       0.0   
1                0.0                    0.0              0.0       0.0   
2                0.0                    0.0              0.0       0.0   
3                0.0                    0.0              0.0       0.0   
4                0.0                    0.0              0.0       0.0   
..               ...                   

In [14]:
toronto_grouped.shape

(96, 273)

In [15]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0                     Lounge   0.2
1             Breakfast Spot   0.2
2               Skating Rink   0.2
3  Latin American Restaurant   0.2
4             Clothing Store   0.2


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.29
1        Pharmacy  0.14
2             Gym  0.14
3  Sandwich Place  0.14
4             Pub  0.14


----Bathurst Manor, Wilson Heights, Downsview North----
                venue  freq
0         Coffee Shop  0.10
1                Bank  0.10
2  Chinese Restaurant  0.05
3    Sushi Restaurant  0.05
4         Gas Station  0.05


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1                 Café  0.25
2   Chinese Restaurant  0.25
3                 Bank  0.25
4                Motel  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0      Sandwich Place  0.09
1  Italian Restaurant  0.09
2         Coffee Shop  0.09
3            

4  Mexican Restaurant  0.00


----Lawrence Manor, Lawrence Heights----
                    venue  freq
0          Clothing Store  0.38
1           Women's Store  0.08
2             Coffee Shop  0.08
3             Event Space  0.08
4  Furniture / Home Store  0.08


----Lawrence Park----
                venue  freq
0         Swim School  0.33
1                Park  0.33
2            Bus Line  0.33
3         Yoga Studio  0.00
4  Mexican Restaurant  0.00


----Leaside----
                 venue  freq
0          Coffee Shop  0.09
1  Sporting Goods Shop  0.09
2                 Bank  0.06
3         Burger Joint  0.06
4        Shopping Mall  0.06


----Little Portugal, Trinity----
                   venue  freq
0                    Bar  0.09
1            Coffee Shop  0.07
2  Vietnamese Restaurant  0.04
3            Men's Store  0.04
4       Asian Restaurant  0.04


----Malvern, Rouge----
                             venue  freq
0             Fast Food Restaurant   1.0
1                    Metr

                             venue  freq
0                             Park   0.5
1                Convenience Store   0.5
2                      Yoga Studio   0.0
3                    Metro Station   0.0
4  Molecular Gastronomy Restaurant   0.0


----York Mills, Silver Hills----
                             venue  freq
0              Martial Arts School   1.0
1                      Yoga Studio   0.0
2                    Metro Station   0.0
3  Molecular Gastronomy Restaurant   0.0
4       Modern European Restaurant   0.0




### Create dataframe

In [38]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [46]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                                        Agincourt                Lounge   
1                           Alderwood, Long Branch           Pizza Place   
2  Bathurst Manor, Wilson Heights, Downsview North           Coffee Shop   
3                                  Bayview Village                  Café   
4                Bedford Park, Lawrence Manor East           Coffee Shop   

  2nd Most Common Venue      3rd Most Common Venue 4th Most Common Venue  \
0          Skating Rink  Latin American Restaurant        Breakfast Spot   
1                   Gym                   Pharmacy           Coffee Shop   
2                  Bank          Mobile Phone Shop           Bridal Shop   
3   Japanese Restaurant         Chinese Restaurant                  Bank   
4        Sandwich Place         Italian Restaurant      Greek Restaurant   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0        Clothing Store             Drugstore        Discount Store   
1        Sandwich Place                   Pub         Women's Store   
2        Sandwich Place                 Diner            Restaurant   
3         Women's Store        Discount Store   Distribution Center   
4      Sushi Restaurant              Pharmacy           Pizza Place   

  8th Most Common Venue 9th Most Common Venue     10th Most Common Venue  
0   Distribution Center               Dog Run           Doner Restaurant  
1               Dog Run    Dim Sum Restaurant                      Diner  
2         Deli / Bodega           Supermarket  Middle Eastern Restaurant  
3               Dog Run      Doner Restaurant                 Donut Shop  
4                   Pub                  Café                    Butcher

### Cluster Neighborhoods

In [47]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [48]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_geo_data

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
#toronto_merged.drop([16], inplace=True)
toronto_merged.reset_index()


toronto_merged

Latitude  Longitude        Borough  \
Postal Code                                        
M1B          43.806686 -79.194353  Scarborough\n   
M1C          43.784535 -79.160497  Scarborough\n   
M1E          43.763573 -79.188711  Scarborough\n   
M1G          43.770992 -79.216917  Scarborough\n   
M1H          43.773136 -79.239476  Scarborough\n   
...                ...        ...            ...   
M9N          43.706876 -79.518188         York\n   
M9P          43.696319 -79.532242    Etobicoke\n   
M9R          43.688905 -79.554724    Etobicoke\n   
M9V          43.739416 -79.588437    Etobicoke\n   
M9W          43.706748 -79.594054    Etobicoke\n   

                                                  Neighborhood  \
Postal Code                                                      
M1B                                             Malvern, Rouge   
M1C                     Rouge Hill, Port Union, Highland Creek   
M1E                          Guildwood, Morningside, West Hill   
M1G                                                     Woburn   
M1H                                                  Cedarbrae   
...                                                        ...   
M9N                                                     Weston   
M9P                                                  Westmount   
M9R          Kingsview Village, St. Phillips, Martin Grove ...   
M9V          South Steeles, Silverstone, Humbergate, Jamest...   
M9W                        Northwest, West Humber - Clairville   

             Cluster Labels       1st Most Common Venue 2nd Most Common Venue  \
Postal Code                                                                     
M1B                     4.0        Fast Food Restaurant             Drugstore   
M1C                     1.0  Construction & Landscaping                   Bar   
M1E                     1.0                  Restaurant        Medical Center   
M1G                     1.0                 Coffee Shop    Mexican Restaurant   
M1H                     1.0            Hakka Restaurant    Athletics & Sports   
...                     ...                         ...                   ...   
M9N                     0.0                        Park         Women's Store   
M9P                     1.0                 Pizza Place        Discount Store   
M9R                     1.0                        Park           Pizza Place   
M9V                     1.0               Grocery Store           Pizza Place   
M9W                     1.0                         Bar         Garden Center   

             3rd Most Common Venue 4th Most Common Venue  \
Postal Code                                                
M1B                          Diner        Discount Store   
M1C                  Women's Store   Dumpling Restaurant   
M1E                   Intersection    Mexican Restaurant   
M1G          Korean BBQ Restaurant         Women's Store   
M1H                         Bakery           Gas Station   
...                            ...                   ...   
M9N                      Drugstore                 Diner   
M9P                    Coffee Shop        Sandwich Place   
M9R                       Bus Line        Sandwich Place   
M9V                   Liquor Store              Pharmacy   
M9W            Rental Car Location             Drugstore   

            5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
Postal Code                                                                     
M1B           Distribution Center               Dog Run      Doner Restaurant   
M1C           Distribution Center               Dog Run      Doner Restaurant   
M1E                          Bank   Rental Car Location        Breakfast Spot   
M1G                     Drugstore        Discount Store   Distribution Center   
M1H          Caribbean Restaurant       Thai Restaurant                  Bank   
...                           ...                   ...                   ..

In [52]:
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].fillna(0)

In [53]:

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)



# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [54]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Longitude 1st Most Common Venue       2nd Most Common Venue  \
Postal Code                                                                
M1V         -79.284577                  Park                  Playground   
M1X         -79.205636                   NaN                         NaN   
M2M         -79.408493                   NaN                         NaN   
M2P         -79.400049                  Park           Convenience Store   
M3A         -79.329656     Food & Drink Shop                        Park   
M4J         -79.338106          Intersection                        Park   
M4N         -79.388790                  Park                 Swim School   
M4W         -79.377529                  Park                  Playground   
M6E         -79.453512                  Park               Women's Store   
M6L         -79.490074                  Park  Construction & Landscaping   
M9A         -79.532242                   NaN                         NaN   
M9N         -79.518188                  Park               Women's Store   

            3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
Postal Code                                                                     
M1V                        Bakery            Donut Shop                 Diner   
M1X                           NaN                   NaN                   NaN   
M2M                           NaN                   NaN                   NaN   
M2P                 Women's Store             Drugstore                 Diner   
M3A                     Drugstore                 Diner        Discount Store   
M4J             Convenience Store             Drugstore                 Diner   
M4N                      Bus Line            Donut Shop        Discount Store   
M4W                         Trail           Escape Room  Ethiopian Restaurant   
M6E                          Pool            Donut Shop    Dim Sum Restaurant   
M6L                        Bakery         Women's Store             Drugstore   
M9A                           NaN                   NaN                   NaN   
M9N                     Drugstore                 Diner        Discount Store   

            6th Most Common Venue        7th Most Common Venue  \
Postal Code                                                      
M1V                Discount Store          Distribution Center   
M1X                           NaN                          NaN   
M2M                           NaN                          NaN   
M2P                Discount Store          Distribution Center   
M3A           Distribution Center                      Dog Run   
M4J                Discount Store          Distribution Center   
M4N           Distribution Center                      Dog Run   
M4W             Electronics Store  Eastern European Restaurant   
M6E                         Diner               Discount Store   
M6L                Discount Store          Distribution Center   
M9A                           NaN                          NaN   
M9N           Distribution Center                      Dog Run   

            8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
Postal Code                                                                     
M1V                       Dog Run      Doner Restaurant              Drugstore  
M1X                           NaN                   NaN                    NaN  
M2M                           NaN                   NaN                    NaN  
M2P                       Dog Run      Doner Restaurant             Donut Shop  
M3A              Doner Restaurant            Donut Shop          Women's Store  
M4J                       Dog Run      Doner Restaurant             Donut Shop  
M4N              Doner Restaurant             Drugstore     Dim Sum Restaurant  
M4W           Dumpling Restaurant             Drugstore       Department Store  
M6E           Distribution Center               Dog Run       Doner Restaurant

In [55]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Longitude       1st Most Common Venue 2nd Most Common Venue  \
Postal Code                                                                
M1C         -79.160497  Construction & Landscaping                   Bar   
M1E         -79.188711                  Restaurant        Medical Center   
M1G         -79.216917                 Coffee Shop    Mexican Restaurant   
M1H         -79.239476            Hakka Restaurant    Athletics & Sports   
M1J         -79.239476                  Smoke Shop            Playground   
...                ...                         ...                   ...   
M9M         -79.532242              Baseball Field   Dumpling Restaurant   
M9P         -79.532242                 Pizza Place        Discount Store   
M9R         -79.554724                        Park           Pizza Place   
M9V         -79.588437               Grocery Store           Pizza Place   
M9W         -79.594054                         Bar         Garden Center   

             3rd Most Common Venue 4th Most Common Venue  \
Postal Code                                                
M1C                  Women's Store   Dumpling Restaurant   
M1E                   Intersection    Mexican Restaurant   
M1G          Korean BBQ Restaurant         Women's Store   
M1H                         Bakery           Gas Station   
M1J                  Jewelry Store         Women's Store   
...                            ...                   ...   
M9M                 Discount Store   Distribution Center   
M9P                    Coffee Shop        Sandwich Place   
M9R                       Bus Line        Sandwich Place   
M9V                   Liquor Store              Pharmacy   
M9W            Rental Car Location             Drugstore   

            5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
Postal Code                                                                     
M1C           Distribution Center               Dog Run      Doner Restaurant   
M1E                          Bank   Rental Car Location        Breakfast Spot   
M1G                     Drugstore        Discount Store   Distribution Center   
M1H          Caribbean Restaurant       Thai Restaurant                  Bank   
M1J                    Donut Shop                 Diner        Discount Store   
...                           ...                   ...                   ...   
M9M                       Dog Run      Doner Restaurant            Donut Shop   
M9P                  Intersection    Chinese Restaurant         Women's Store   
M9R                       Dog Run    Dim Sum Restaurant                 Diner   
M9V          Fast Food Restaurant            Beer Store   Fried Chicken Joint   
M9W                 Women's Store                 Diner        Discount Store   

            8th Most Common Venue 9th Most Common Venue  \
Postal Code                                               
M1C                    Donut Shop             Drugstore   
M1E             Electronics Store             Drugstore   
M1G                       Dog Run      Doner Restaurant   
M1H           Fried Chicken Joint               Dog Run   
M1J           Distribution Center               Dog Run   
...                           ...                   ...   
M9M                     Drugstore         Women's Store   
M9P                       Dog Run    Dim Sum Restaurant   
M9R                Discount Store   Distribution Center   
M9V                Sandwich Place         Women's Store   
M9W           Distribution Center               Dog Run   

                  10th Most Common Venue  
Postal Code                               
M1C          Eastern European Restaurant  
M1E                           Donut Shop  
M1G                           Donut Shop  
M1H                  Distribution Center  
M1J                     Doner Restaurant  
...                                  ...  
M9M                 Fast Food Restaurant  
M9P                               

In [56]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Longitude 1st Most Common Venue 2nd Most Common Venue  \
Postal Code                                                          
M2L         -79.374714   Martial Arts School         Women's Store   

            3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
Postal Code                                                                     
M2L                     Drugstore                 Diner        Discount Store   

            6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
Postal Code                                                                     
M2L           Distribution Center               Dog Run      Doner Restaurant   

            9th Most Common Venue 10th Most Common Venue  
Postal Code                                               
M2L                    Donut Shop    Dumpling Restaurant

In [57]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Longitude 1st Most Common Venue 2nd Most Common Venue  \
Postal Code                                                          
M9B         -79.554724            Print Shop         Women's Store   

            3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
Postal Code                                                                     
M9B                         Diner        Discount Store   Distribution Center   

            6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
Postal Code                                                                     
M9B                       Dog Run      Doner Restaurant            Donut Shop   

            9th Most Common Venue 10th Most Common Venue  
Postal Code                                               
M9B                     Drugstore           Dessert Shop

In [58]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Longitude 1st Most Common Venue 2nd Most Common Venue  \
Postal Code                                                          
M1B         -79.194353  Fast Food Restaurant             Drugstore   

            3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
Postal Code                                                                     
M1B                         Diner        Discount Store   Distribution Center   

            6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
Postal Code                                                                     
M1B                       Dog Run      Doner Restaurant            Donut Shop   

            9th Most Common Venue 10th Most Common Venue  
Postal Code                                               
M1B           Dumpling Restaurant            Wings Joint